In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Current work
print(os.getcwd())
# Change directory 2 level up
os.chdir("..")
os.chdir("..")
os.chdir("..")

print(os.getcwd())
# Look for the file in the directory
print(os.listdir())

/tmp/ipykernel_9149/2635443006.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


/home/jose/Git/NeurIPS-2024-LLM-and-Psychopathology/experiments/experiment_1/notebooks
/home/jose/Git/NeurIPS-2024-LLM-and-Psychopathology
['models', '.gitignore', 'experiments', 'include', '.git', 'README.md', 'LICENSE', 'wat']


In [2]:
data = pd.read_csv(f'experiments/experiment_1/data/self_report_study2.csv')

data.head()

,Unnamed: 0,subj,age,iq,gender,lsas_total,bis_total,sds_total,scz_total,aes_total,stai_total,eat_total,audit_total,oci_total,Factor1,Factor2,Factor3
0,1,304SM51WA34XCM4E67SL28LY2UCBS4,22,85.742035,-1,68,77,44,29,30,50,5,5,27,0.599498,1.004987,0.592111
1,2,308Q0PEVB8DPUFML3BKB70KL51T9IW,27,115.266850,1,64,72,37,22,33,38,3,2,15,0.069899,-0.318654,0.536820
2,3,308Q0PEVB8DPUFML3BKB70KL6Q8I9Z,30,93.811195,1,26,55,26,13,26,31,4,12,15,-1.058884,-0.234499,-0.377010
3,4,308XBLVESI4LB741RISGCYG2G16RBR,25,108.106105,1,17,57,38,16,31,30,12,0,11,-0.704472,0.139747,-0.811099
4,5,30BUDKLTXDVUDNIN42H3A5HRS51E5M,33,95.588980,-1,43,74,51,17,48,53,19,1,28,1.095160,0.180707,0.182847


In [3]:
pathology_columns = {
    'Depression': 'sds_total',
    'Trait Anxiety': 'stai_total',
    'Eating Disorder': 'eat_total',
    'Alcohol Addiction': 'audit_total',
    'Impulsivity': 'bis_total',
    'Schizophrenia': 'scz_total',
    'Obsessive Compulsive Disorder': 'oci_total',
    'Apathy': 'aes_total',
    'Social Anxiety': 'lsas_total',
}

ranges = {
    'Depression': (20, 80),
    'Trait Anxiety': (20, 80),
    'Eating Disorder': (0, 78),
    'Alcohol Addiction': (0, 40),
    'Impulsivity': (30, 120),
    'Schizophrenia': (0, 43),
    'Obsessive Compulsive Disorder': (0, 60),
    'Apathy': (18, 72),
    'Social Anxiety': (0, 144)
}


def normal_scoring(scoring_range: tuple, scoring_value: int) -> float:
    min_val, max_val = scoring_range
    normal_result = (scoring_value - min_val) / (max_val - min_val)
    return normal_result

for pathology, column in pathology_columns.items():
    range_ = ranges[pathology]
    data[column] = data[column].apply(lambda x: normal_scoring(range_, x))

In [4]:
thresholds = {
    'Depression': 0.5,
    'Trait Anxiety': 0.3,
    'Eating Disorder': 0.256,
    'Alcohol Addiction': 0.34,
    'Impulsivity': 0.467,
    'Schizophrenia': 0.461,
    'Obsessive Compulsive Disorder': 0.399,
    'Apathy': 0.389,
    'Social Anxiety': 0.444
}

pathology_results = {pathology: [] for pathology in list(pathology_columns.keys()) + ['No Pathology']}

for pathology, column in pathology_columns.items():
    pathological = data[data[column] > thresholds[pathology]].nlargest(1080, column)
    pathology_results[pathology] = pathological[pathology_columns.values()].values.tolist()

ordered_columns = list(pathology_columns.values())
ordered_thresholds = [thresholds[pathology] for pathology in pathology_columns.keys()]
non_pathological_criteria = (data[ordered_columns] <= pd.Series(ordered_thresholds, index=ordered_columns)).all(axis=1)

non_pathological = data[non_pathological_criteria].sample(n=min(100, sum(non_pathological_criteria)))
pathology_results['No Pathology'] = non_pathological[ordered_columns].values.tolist()

lengths = {pathology: len(vectors) for pathology, vectors in pathology_results.items()}
lengths

{'Depression': 257,
 'Trait Anxiety': 830,
 'Eating Disorder': 150,
 'Alcohol Addiction': 110,
 'Impulsivity': 225,
 'Schizophrenia': 334,
 'Obsessive Compulsive Disorder': 263,
 'Apathy': 233,
 'Social Anxiety': 351,
 'No Pathology': 100}

In [5]:
pathology_results

{'Depression': [[0.85,
   0.9166666666666666,
   0.3333333333333333,
   0.0,
   0.24444444444444444,
   0.5581395348837209,
   0.95,
   0.3888888888888889,
   0.7847222222222222],
  [0.8333333333333334,
   0.8833333333333333,
   0.1282051282051282,
   0.075,
   0.4666666666666667,
   0.4883720930232558,
   0.18333333333333332,
   0.3888888888888889,
   0.8055555555555556],
  [0.8166666666666667,
   0.95,
   0.4230769230769231,
   0.525,
   0.7555555555555555,
   0.813953488372093,
   0.25,
   0.5185185185185185,
   0.4513888888888889],
  [0.8166666666666667,
   0.85,
   0.34615384615384615,
   0.0,
   0.35555555555555557,
   0.5581395348837209,
   0.8333333333333334,
   0.2777777777777778,
   0.6736111111111112],
  [0.8,
   0.9,
   0.0641025641025641,
   0.175,
   0.4888888888888889,
   0.7209302325581395,
   0.5833333333333334,
   0.4444444444444444,
   0.5972222222222222],
  [0.8,
   0.85,
   0.0641025641025641,
   0.0,
   0.3333333333333333,
   0.3488372093023256,
   0.3833333333333

In [6]:
csv_columns = ['Depression', 'Trait Anxiety', 'Eating Disorder', 'Alcohol Addiction',
               'Impulsivity', 'Schizophrenia', 'Obsessive Compulsive Disorder',
               'Apathy', 'Social Anxiety', 'Induced Pathology', 'Iteration']

df_csv = pd.DataFrame(columns=csv_columns)
for i in range(100):
    for pathology, vectors in pathology_results.items():
        row = vectors[i] + [pathology, i+1]
        df_csv.loc[len(df_csv)] = row

In [7]:
def sort_group(group):
    no_pathology_row = group[group['Induced Pathology'] == 'No Pathology']
    other_rows = group[group['Induced Pathology'] != 'No Pathology']
    return pd.concat([no_pathology_row, other_rows])

data_sorted_correctly = df_csv.groupby('Iteration').apply(sort_group).reset_index(drop=True)

/tmp/ipykernel_9149/1976438453.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_sorted_correctly = df_csv.groupby('Iteration').apply(sort_group).reset_index(drop=True)


In [8]:
data_sorted_correctly

,Depression,Trait Anxiety,Eating Disorder,Alcohol Addiction,Impulsivity,Schizophrenia,Obsessive Compulsive Disorder,Apathy,Social Anxiety,Induced Pathology,Iteration
0,0.100000,0.133333,0.012821,0.000,0.400000,0.232558,0.083333,0.055556,0.131944,No Pathology,1
1,0.850000,0.916667,0.333333,0.000,0.244444,0.558140,0.950000,0.388889,0.784722,Depression,1
2,0.750000,1.000000,0.205128,0.050,0.611111,0.813953,0.200000,0.666667,0.812500,Trait Anxiety,1
3,0.366667,0.466667,0.948718,0.025,0.288889,0.325581,0.583333,0.314815,0.791667,Eating Disorder,1
4,0.333333,0.616667,0.358974,0.850,0.600000,0.558140,0.566667,0.444444,0.506944,Alcohol Addiction,1
...,...,...,...,...,...,...,...,...,...,...,...
995,0.433333,0.750000,0.256410,0.025,0.544444,0.418605,0.533333,0.351852,0.673611,Impulsivity,100
996,0.666667,0.850000,0.243590,0.250,0.377778,0.604651,0.383333,0.333333,0.201389,Schizophrenia,100
997,0.533333,0.550000,0.025641,0.075,0.266667,0.162791,0.583333,0.185185,0.576389,Obsessive Compulsive Disorder,100
998,0.316667,0.600000,0.025641,0.125,0.422222,0.348837,0.233333,0.500000,0.277778,Apathy,100
